# Useful Applications

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import math

from jupyterthemes import jtplot
jtplot.style(theme='onedork', figsize=(15, 9))

## Roll's model to estimate spread (1984)

In [3]:
df = pd.read_csv('../../data/BK_2002-01-01_2022-12-31_1d.csv', index_col=0, parse_dates=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2001-12-31,26.588255,26.880790,26.490742,26.523245,1106703,0.0,0.0
2002-01-02,26.523270,26.653287,25.879690,26.646786,1397930,0.0,0.0
2002-01-03,26.750778,27.368352,26.750778,27.238337,2267556,0.0,0.0
2002-01-04,27.595880,27.940422,27.472366,27.699894,1658780,0.0,0.0
2002-01-07,28.538502,28.701022,27.953431,28.571007,3413033,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-23,44.103883,44.371842,43.865701,44.312298,2273400,0.0,0.0
2022-12-27,44.471088,44.580253,44.173354,44.481010,2109600,0.0,0.0
2022-12-28,44.481012,44.590180,44.222980,44.292450,2013100,0.0,0.0


In [4]:
close = df['Close'].to_frame()
close

,Close
Date,
2001-12-31,26.523245
2002-01-02,26.646786
2002-01-03,27.238337
2002-01-04,27.699894
2002-01-07,28.571007
...,...
2022-12-23,44.312298
2022-12-27,44.481010
2022-12-28,44.292450


In [15]:
close['Diff'] = df['Close'].diff()
close

,Close,Diff
Date,,
2001-12-31,26.523245,NaN
2002-01-02,26.646786,0.123541
2002-01-03,27.238337,0.591551
2002-01-04,27.699894,0.461557
2002-01-07,28.571007,0.871113
...,...,...
2022-12-23,44.312298,0.168716
2022-12-27,44.481010,0.168713
2022-12-28,44.292450,-0.188560


In [21]:
np.cov(close['Close'], close['Close'].shift(-1))

array([[108.98567575,          nan],
       [         nan,          nan]])

In [34]:
np.cov(close['Diff'][:-1], close['Diff'][1:])[1,1]

0.37847537685170574